# Setup Intelligent Recommendations Service

Intelligent Recommendations is a general purpose service that offers patented capabilities to more effectively drive desired outcomes out of the box such as “shop similar looks,” “shop by description,” “real time,” “session based”, Item based recommendations that can combine User interactions and Item Metadata. Businesses can promote and personalize any content type, such as sellable products, consumable media, documents, videos, and more.

Learn more at [Explore Intelligent Recommendations](https://intelligent-recommendations.microsoft.com/).


## 1. Create & Configure Intelligent Recommendations

You are now ready to setup the Intelligent Recommendation Service!

**1.1 Create Service Account**

Follow steps 1-5 in [Create the service account](https://docs.microsoft.com/en-us/industry/retail/intelligent-recommendations/deploy-intelligent-recommendations-account#create-the-service-account).  You will leave the "Reports Connection String" blank at this point, we will configure it in the next step once the resource is created.

**TIP:** Do add your tenant and user id as described in the Note on Step 4!

**1.2. Configure Logging**

Configure logging using the folder you created in your Azure Storage Container for logs by following the instructions [here](https://docs.microsoft.com/en-us/industry/retail/intelligent-recommendations/setup-error-logging#configure-the-reports-feature).  Note you will link to your folder, not the root.

**TIP:** For our example, the connection string should be similar to (replace *your_storage_account* with your account name): 
&nbsp;&nbsp;&nbsp;&nbsp;*https://your_storage_account.blob.core.windows.net/ircontainer/irlogs*  

**1.3. Add Modeling & Serving Components**

Then complete all the steps in the [Add modeling and serving components](https://docs.microsoft.com/en-us/industry/retail/intelligent-recommendations/deploy-intelligent-recommendations-account#add-modeling-and-serving-components) section.

**TIP:** For the serving component, you can select the minimum number of requests per second (1) for the "Pre-allocated Capacity" to work through this example.

**TIP:** When you create the Modeling component there is an option to select "Feature Set".  We show examples of using the "Premium" features in this notebook, but if you want to maintain a free/lower tier cost for the service, you can select "Basic" or "Standard". 


Once the modeling component is deployed, it will automatically initiate the cooking process that processes the data and creates the recommendations.  This may take anywhere from a few minutes to a few hours.  The best way to monitor progress is to watch for files to be written to the logging folder (e.g. ir_logs in this example).

When the cooking process completes you can view the results at the [Results Explorer Portal](https://docs.microsoft.com/en-us/industry/retail/intelligent-recommendations/results-explorer#step-2-view-results).   Until the process is complete though, you will not see any results when you "Execute Request".

While IR processes, continue with the next steps so you can authenticate and make API calls to the service once it's complete.


## 2. Create Client Application & Configure Authentication

Intelligent Recommendations requires an OAuth2 authentication. Determine (or create) the Active Directory application from which to call the service endpoint with authentication. Any unauthenticated calls will fail with an error.  

To establish this, complete the tasks in [Step 2: Setup Client Application](https://docs.microsoft.com/en-us/industry/retail/intelligent-recommendations/quickstart-endpoint#step-2-set-up-a-client-application).

You can use either a secret or a certificate for authentication, for this example we'll create a secret.  Make sure to note the value for the Client Secret when you create it as we will need it to configure the connections we use in this notebook.

From the application you registered in the previous step, continue with [Step 3: Configure Authentication](https://docs.microsoft.com/en-us/industry/retail/intelligent-recommendations/quickstart-endpoint#step-3-configure-authentication).  


**TIP:** Make sure to note the "Value" for your Client Secret in the previous step as you can't access it after you create it.  If you lose it, add a new client Secret and note the Value before navigating from the page.

You can stop once you get to the "Construct API Request" step as we will do this in this Notebook.




## 3. (Optional) Configure CORS

While this is not required for this example solution accelerator, if you wanted to use a web application in another domain to access the Intelligent Recommendations service, you could configure Cross-Origin Resource Sharing.

Instructions provided [here](https://docs.microsoft.com/en-us/industry/retail/intelligent-recommendations/deploy-intelligent-recommendations-account#configure-cors-optional).

# Setup & Configuration

In [ ]:
import requests
import json
import base64
import pandas as pd

The following fields should be completed based on the Intelligent Recommendations account and the Client Application setup from the Preqrequisites.

In [ ]:
ir_base_uri = 'Your_IR_Service_Endpoint'  #the Url from the overview page for the Service Endpoint in the IR account (Step 1 above)
model = 'Your_Model_Name'  #the name of the modeling component in your IR account

client_id = 'Your_Client_ID'  #the application client id you created in Step 2 of the setup.
client_secret = 'Your_Client_Secret_Value' #from step 2 above.

auth_server_url = "https://login.microsoftonline.com/72f988bf-86f1-41af-91ab-2d7cd011db47/oauth2/v2.0/token"
scope = 'c5b731db-1b0a-43f6-bcf6-757667d9cdc6/.default'

In [ ]:
# Encode the client ID and client secret
authorization = base64.b64encode(bytes(client_id + ":" + client_secret, "ISO-8859-1")).decode("ascii")

headers = {
    "Authorization": f"Basic {authorization}",
    "Content-Type": "application/x-www-form-urlencoded"
}
body = {
    "grant_type": "client_credentials", "scope" : scope
}

response = requests.post(auth_server_url, data=body, headers=headers)
#print(response.text)


tokens = json.loads(response.text)
token = tokens['access_token']
#print(token)

Make a test call to the service to confirm connectivity; this should return a set of "popular" items.

## Test API Call to IR Service

In [ ]:
ir_popular_items = ir_base_uri + '/reco/v1.0/popular?modeling={}'.format(model)

api_call_headers = {'Authorization': 'Bearer ' + token}
api_call_response = requests.get(ir_popular_items, headers=api_call_headers)

print(api_call_response.text)

If the processing has completed, you will get a list of popular items that should look similar to:

```console
{"status":"Success","version":null}
{"error":{"code":"TooManyRequests","message":"Too many requests"}}
{"id":"Lists","name":"Lists","version":"v1.0","interactionsVersion":"20220713231250","items":[{"id":"165","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"103","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"123","trackingId":"00000000-0000-0000-0000-000000000003"},
...
{"id":"41","trackingId":"00000000-0000-0000-0000-000000000003"}],"title":"Popular","longTitle":"Popular","titleId":5,"pagingInfo":{"totalItems":167},"status":"Success"}

```

If instead, you get the following message:

```console
{"error":{"code":"ServiceUnavailable","message":"Service is unavailable"}}
```

Your Intelligent Recommendations Service is likely still processing the data.  You can continue to wait, and monitor the logs by watching for files to be written to the logging folder (e.g. ir_logs in this example).  

Or, you can manually trigger the cooking process by running the following cell.

## (Optional) Trigger Modeling Manually

In [ ]:
ir_trigger_model = ir_base_uri + '/Control/V1.0/Model?modeling={}'.format(model)

api_call_headers = {'Authorization': 'Bearer ' + token}
api_call_response = requests.post(ir_trigger_model, headers=api_call_headers)

print(api_call_response.text)

If the IR cooking process wasn't already running, you will get the message:

```console
{"status":"Success","version":null}
```



If the process was already running (or if you repeat/run again) you will get the message:

```console
{"error":{"code":"TooManyRequests","message":"Too many requests"}}
```

As you can only trigger modeling once in a 24 hour period.

The service will take some time to complete the process, check back every 10 minutes and re-run "Test API Call to IR Service" (above) until you get a list of popular items.

# Load Product Catalog and Orders Data

Once the IR Service has completed processing and you are able to view the list of popular items when you test the API call, proceed with the next steps.

## Product Catalog

We are bringing in the original data (formatted in the 01_Data_Preparation notebook) to help us interpret the results from the Intelligent Recommendation system.

In [ ]:
products_df=pd.read_csv('data/products.csv')
print(products_df.shape)
products_df.head()

## Orders (Interactions)

In [ ]:
orders_df =pd.read_csv('data/orders.csv')
print(orders_df.shape)
orders_df.head()

# Recommendation Functions

## API Calls for Recommendations

The following function constructs the API calls for the different types of recommendations available in the service.

In [ ]:
def recommendation_uri_builder(base_uri, command, model, seed_item = 0, seed_user = 0, return_count = 100, seed_items_list = '', skip_count = 0):

    #as of July 2022, this is the API path
    ir_path = '/reco/v1.0/'

    
    if command == 'similar':  #FOR FUTURE: algo type for similar are Visual, Textual: '/reco/v1.0/similar/{}?AlgoType=Textual&modeling={}&count={}'.format(seed_item, model, return_count)
        uri = ir_base_uri + ir_path + 'similar/{}?modeling={}&count={}'.format(seed_item, model, return_count)
    elif command == 'popular':
        uri = ir_base_uri + ir_path + 'popular?modeling={}&count={}'.format(model, return_count)
    elif command == 'new':
        uri = ir_base_uri + ir_path + 'new?modeling={}&count={}'.format(model, return_count)
    elif command == 'trending':
        uri = ir_base_uri + ir_path + 'trending?modeling={}&count={}'.format(model, return_count)
    elif command == 'picks':
        #algo types for user picks is only RecentPurchases
        uri = ir_base_uri + ir_path + 'picks?AlgoType=RecentPurchases&modeling={}&userId={}&count={}'.format(model, seed_user, return_count)
    elif command == 'fbt':
        uri = ir_base_uri + ir_path + 'cart/{}?modeling={}&count={}'.format(seed_item, model, return_count)
     elif command == 'cart':
        uri = ir_base_uri + ir_path + 'cart/items?seedItemIds={}&modeling={}&count={}&skipItems={}&userId={}'.format(seed_items_list, model, return_count, skip_count, seed_user)
    else:
        print("No URI could be formed")


    return uri

In [ ]:
def return_recommended_items(uri, token):
    api_call_headers = {'Authorization': 'Bearer ' + token}
    api_call_response = requests.get(uri, headers=api_call_headers)  

    rec_items = pd.json_normalize(json.loads(api_call_response.text)['items'])['id']

    return rec_items


# Top Charts

## Popular Items

The most popular items from the catalog are ranked by the number of interactions over a period of time.  We request the 25 most popular items then review the descriptions in our product catalog.  

In [ ]:
popular_uri = recommendation_uri_builder(ir_base_uri, 'popular', model, return_count=25)
popular_items = return_recommended_items(popular_uri, token)

In [ ]:
print("Most Popular Items")
products_df[products_df['itemId'].isin(popular_items)]

In [ ]:
# See the count of orders (baskets) that contain the popular items
popular_baskets_df = orders_df[orders_df['itemId'].isin(popular_items)][['itemDescription', 'basketId']].groupby('itemDescription').count()
popular_baskets_df.reset_index(inplace=True)
popular_baskets_df.columns = ['itemDescription', 'countOfOrders']
popular_baskets_df.sort_values(by=['countOfOrders'], ascending=False)

## Trending Items

Returns the catalog items ranked by popularity, giving newer events more weight than older ones. Popularity is scored over a configurable period of time.

In [ ]:
trending_uri = recommendation_uri_builder(ir_base_uri, 'trending', model, return_count=10)
trending_items = return_recommended_items(trending_uri, token)

In [ ]:
print("Trending Items")
products_df[products_df['itemId'].isin(trending_items)]

## New Items

Returns the catalog items ranked by release date, newest first.



In [ ]:
new_uri = recommendation_uri_builder(ir_base_uri, 'new', model, return_count=15)
new_items = return_recommended_items(new_uri, token)

In [ ]:
products_df[products_df['itemId'].isin(new_items)]

# Item-to-Item Recommendations

## Cart Recommendations

Similar to frequently bought together, but instead of a single item, can provide a list of items.



In [ ]:
# NOTE: no spaces in the string of item ids 
baking_items = '16,37,50'  #butter, cooking chocolate, eggs
item_count = 5

cart_picks_uri = recommendation_uri_builder(ir_base_uri, 'cart', model, seed_items_list = baking_items, return_count = item_count)
cart_pick_items = return_recommended_items(cart_picks_uri, token)

In [ ]:
# The seed items 
int_list = [int(s) for s in baking_items.split(',')]
products_df[products_df['itemId'].isin(int_list)]

In [ ]:
#Recommendations
products_df[products_df['itemId'].isin(cart_pick_items)]

## (Optional) Similar Items

Returns a list of item-to-item recommendations. Recommended items can be similar to the seed item or related in some way, based on the way data is structured and the algorithm.  

**Note:** This option is only available if you selected the "Premium" feature set for your modeling component.

Because we only have have product titles in the example dataset, these results are likely not as meaningful as they would be if we had a textual description (or images) available.  

In [ ]:
similar_seed_id = 42  # curd cheese
item_count = 10

similar_uri = recommendation_uri_builder(ir_base_uri, 'similar', model, seed_item = similar_seed_id, return_count = item_count)
similar_items = return_recommended_items(similar_uri, token)

In [ ]:
print("Original item")
products_df[products_df['itemId']==similar_seed_id]

In [ ]:
products_df[products_df['itemId'].isin(similar_items)]

## (Optional) Frequently Bought Together

Returns items that are frequently bought along with the seed item.

**NOTE:** This option is only available if you select the "Premium" feature set for your modeling component.


In [ ]:
item_seed_id = 21  # butter = 16, canned beer 21
item_count = 10

fbt_picks_uri = recommendation_uri_builder(ir_base_uri, 'fbt', model, seed_item = item_seed_id, return_count = item_count)
fbt_pick_items = return_recommended_items(fbt_picks_uri, token)

In [ ]:
print("Original item")
products_df[products_df['itemId']==item_seed_id]

In [ ]:
products_df[products_df['itemId'].isin(fbt_pick_items)]

The following shows the difference between 1 item (Cart) and multiple items (Frequently Bought Together).

In [ ]:
party_items =  '129'  #12 beer, 129 salty snack, 162 whisky

cart_picks_uri = recommendation_uri_builder(ir_base_uri, 'cart', model, seed_items_list = party_items, return_count = item_count)
cart_pick_items = return_recommended_items(cart_picks_uri, token)

In [ ]:
# The seed items 
int_list = [int(s) for s in party_items.split(',')]
products_df[products_df['itemId'].isin(int_list)]

In [ ]:
products_df[products_df['itemId'].isin(cart_pick_items)]

Now, we will add a drink and see how it changes things slightly.

In [ ]:
# NOTE: no spaces in the string of item ids 
party_items =  '129,139' 

cart_picks_uri = recommendation_uri_builder(ir_base_uri, 'cart', model, seed_items_list = party_items, return_count = item_count)
cart_pick_items = return_recommended_items(cart_picks_uri, token)

In [ ]:
products_df[products_df['itemId'].isin(cart_pick_items)]

# User-to-Item Recommendations

## (Optional) User Picks
Returns personal picks, or items from the catalog that the user is likely to interact with based on their historical picks.

**NOTE:** This option is only available if you select the "Premium" feature set for your modeling component.

In [ ]:
user_seed_id = 1000 
item_count = 5

user_picks_uri = recommendation_uri_builder(ir_base_uri, 'picks', model, seed_user = user_seed_id, return_count = item_count)
user_pick_items = return_recommended_items(user_picks_uri, token)

View their previous order history.

In [ ]:
orders_df[orders_df['Member_number'] == user_seed_id]

The recommendations based on that history.

In [ ]:
products_df[products_df['itemId'].isin(user_pick_items)]